<a href="https://colab.research.google.com/github/jackashore/judgement/blob/develop/Ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import nltk
import pandas as pd
from bs4 import BeautifulSoup
from string import punctuation
from pymorphy2 import MorphAnalyzer
from nltk.collocations import *
from nltk.corpus import stopwords
from nltk.util import everygrams
from collections import Counter, OrderedDict
from sklearn.feature_extraction.text import TfidfVectorizer

stops = set(stopwords.words('russian'))
punct = punctuation+'«»—…“”*№–'
morph = MorphAnalyzer()

In [2]:
# nltk.download('stopwords')

In [3]:
df = pd.DataFrame()

In [4]:
# Функции скопированы из тетрадки про парсинг xml, со временем это превратится в прекрасного лебедя (импортируемый модуль)

meta_data = {'court': '', 'judge': '', 'prosecutor': '', 'secretary': '', 'accused': '', 'result': '', 'category': '', 
            'punishment_type': '', 'punishment_term': ''}

def parse_xml(path):   # парсим xml
    with open(path, 'r', encoding='utf-8')as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        
        meta_data['court'] = soup.court.string     # добавляем в метаданные то, что имеется в разметке
        meta_data['judge'] = soup.judge.string
        meta_data['result'] = soup.result.string
        meta_data['category'] = soup.category.string
        
        html = []
        for line in soup.body:
            line = line.replace('[', '')
            line = line.replace(']', '') 
            html.append(line)
        true_html = ' '.join(html)
                
        html_soup = BeautifulSoup(true_html, 'html.parser')
        return html_soup.get_text()
    
def get_parts(text):    # делим на части
    lines = [line for line in text.split('\n')]
    beg, end = 0, 0 
    for num, line in enumerate(lines):
        if 'установил' in line.lower() or 'у с т а н о в и л' in line.lower(): 
            beg = num + 1
        if 'приговорил' in line.lower() or 'п р и г о в о р и л' in line.lower() or 'постановил'\
         in line.lower() or 'п о с т а н о в и л' in line.lower():
            end = num + 1
        
        if beg and end:
            beginning = [stroka.strip() for stroka in lines[:beg]]
            main_part = [stroka.strip() for stroka in lines[beg:end]]
            ending = [stroka.strip() for stroka in lines[end:]]
            return [' '.join(main_part), ' '.join(beginning), ' '.join(ending)]
        
def clean(text):
    pattern1 = re.compile(r'(п|ч|ст)(\.|\s|\d)')
    pattern2 = re.compile(r'((У|Г)П?К|КоАП|ПДД)\sРФ')
    pattern3 = re.compile('\d')
    pattern4 = re.compile('(ДД.ММ.ГГ|дд.мм.гг)')

    text = re.sub(pattern1, ' ABBR ', text)
    text = re.sub(pattern2, ' DOCUMENT ', text)
    text = re.sub(pattern3, ' DIGIT ', text)
    text = re.sub(pattern4, ' DATE ', text)
    text = text.replace('\xad', '')
    text = text.replace('п р и г о в о р и л', '')
    text = text.replace('п о с т а н о в и л', '')
    return text
        
def process_xml(path):
    return clean(get_parts(parse_xml(path))[0])

In [5]:
main_path = r'C:\Users\a.reshetniko\Documents\trash\judgment\judgement\belinskij-rajonnyj-sud-penzenskaya-oblast-263'
raw_texts = []
for items in os.walk(main_path):
    for file in items[2]:
        file_path = os.path.join(items[0], file)
        raw_texts.append(process_xml(file_path))

df['raw_main_parts'] = raw_texts

* Прикрутить Верин токенизатор
* Подумать о лемматизаторе

In [6]:
def normalize(text):
    trash = ['abbr', 'document', 'digit', 'date']
    words = [word.strip(punct) for word in text.lower().split()] 
    words = [word for word in words if word and word not in trash]
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

In [7]:
df['norm_main_parts'] = df['raw_main_parts'].apply(normalize)

In [8]:
df.head()

,raw_main_parts,norm_main_parts
0,"Стульников С.В. совершил кражу, т.е. тайное х...","[стульник, с.в, совершить, кража, т.е, тайный,..."
1,"Крутилин С.В., являясь лицом, управляющим авт...","[крутилина, с.в, являться, лицо, управлять, ав..."
2,Видюлин П.А. обвиняется в совершении неправом...,"[видюлина, п.а, обвиняться, в, совершение, неп..."
3,"Кузнецов М.Ю. совершил нарушение лицом, управ...","[кузнецов, м.ю, совершить, нарушение, лицо, уп..."
4,"Кочетков Д.Г. совершил нарушение лицом, управ...","[кочетков, д.г, совершить, нарушение, лицо, уп..."


In [9]:
df = df.dropna()

In [10]:
all_t = []
for text in df.norm_main_parts:
    all_t.extend(text)

In [11]:
colloc_1000 = []
for col in Counter(list(everygrams(all_t, min_len=3, max_len=10))).most_common(1000):
    colloc_1000.append(' '.join(col[0]))

In [12]:
tfidf = TfidfVectorizer(ngram_range=(3,10), min_df=10)

In [13]:
df['tokens_str'] = [' '.join(i) for i in df.norm_main_parts]

In [14]:
texts_vectors = tfidf.fit_transform(df['tokens_str'])

In [15]:
texts_vectors

<115x3913 sparse matrix of type '<class 'numpy.float64'>'
	with 88423 stored elements in Compressed Sparse Row format>

In [16]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [17]:
df['tf-idf'] = pd.Series()
df['tf-idf'] = df['tf-idf'].astype(object)

In [18]:
some_array = []
for i in range(len(df.tokens_str)):
    some_dict = {}
    for num, s in enumerate(texts_vectors.toarray()[i]):
        some_dict.setdefault((id2word[num]), s)
    sorted_dict = OrderedDict(sorted(some_dict.items(), key=lambda t: t[1])[::-1])    # сортировка значений по убыванию
    minim_array = [item[0] for item in sorted_dict.items() if item[1]!=0]
    df.iat[i,3] = minim_array

In [19]:
df['jur'] = pd.Series()
df['jur'] = df['jur'].astype(object)
for num, item in enumerate(df['tf-idf']):
    df.iloc[num, 4] = set(colloc_1000) & set(item)

In [20]:
df

,raw_main_parts,norm_main_parts,tokens_str,tf-idf,jur
0,"Стульников С.В. совершил кражу, т.е. тайное х...","[стульник, с.в, совершить, кража, т.е, тайный,...",стульник с.в совершить кража т.е тайный хищени...,"[фио пояснить судебный заседание следующий, фи...","{при определение вид, тайный хищение чужое иму..."
1,"Крутилин С.В., являясь лицом, управляющим авт...","[крутилина, с.в, являться, лицо, управлять, ав...",крутилина с.в являться лицо управлять автомоби...,"[на проезжий часть, фио пояснить судебный засе...",{тяжесть вред причинить здоровье человек прика...
2,Видюлин П.А. обвиняется в совершении неправом...,"[видюлина, п.а, обвиняться, в, совершение, неп...",видюлина п.а обвиняться в совершение неправоме...,"[иметь умысел на, учёт весь обстоятельство дел...","{полностью осознать последствие, что он полнос..."
3,"Кузнецов М.Ю. совершил нарушение лицом, управ...","[кузнецов, м.ю, совершить, нарушение, лицо, уп...",кузнецов м.ю совершить нарушение лицо управлят...,"[ход свой движение, по ход свой движение, по х...","{тяжкий вред здоровье по признак, тяжесть вред..."
4,"Кочетков Д.Г. совершил нарушение лицом, управ...","[кочетков, д.г, совершить, нарушение, лицо, уп...",кочетков д.г совершить нарушение лицо управлят...,"[на место дтп, судебный заседание показать что...","{тяжкий вред здоровье по признак, тяжесть вред..."
5,Горшков Д.А. обвиняется в причинении смерти п...,"[горшков, д.а, обвиняться, в, причинение, смер...",горшков д.а обвиняться в причинение смерть по ...,"[учёт который суд, судебный разбирательство го...","{полностью осознать последствие, что он полнос..."
6,"Сойнов Н.Ю. обвиняется в совершении кражи, то ...","[сойновый, н.ю, обвиняться, в, совершение, кра...",сойновый н.ю обвиняться в совершение кража то ...,"[проведение судебный разбирательство, приговор...","{полностью осознать последствие, что он полнос..."
7,Седов А.С. и Полиневский В.В. обвиняются в со...,"[седов, а.с, и, полиневский, в.в, обвиняться, ...",седов а.с и полиневский в.в обвиняться в совер...,"[по предварительный сговор, лицо по предварите...","{умысел на тайный хищение, по место жительство..."
8,"Филин В.Г. совершил грабеж, т.е. открытое хище...","[филин, в.г, совершить, грабёж, т.е, открытый,...",филин в.г совершить грабёж т.е открытый хищени...,"[опасный для жизнь, через некоторый время, при...","{при определение вид, через некоторый время, х..."
9,Яшин Н.Н. и Творогов Р.А. обвиняются в соверше...,"[яшин, н.н, и, творог, р.а, обвиняться, в, сов...",яшин н.н и творог р.а обвиняться в совершение ...,"[опасный для жизнь, проведение судебный разбир...","{обстоятельство существенно уменьшать, посколь..."


In [21]:
smth = ' '.join(df['norm_main_parts'][0])
for j in df['jur'][0]:
    smth = smth.replace(j, 'JUR')

In [22]:
smth

'стульник с.в совершить кража т.е JUR с незаконный проникновение в иной хранилище JUR в ночь на в цель JUR действовать JUR стульник с.в прийти к домовладение по в принадлежащий фио где открыть рука запорный устройство на входной дверь и незаконно проникнуть в сарай расположить на территория данный домовладение JUR один гусь стоимость рубль после что с место преступление скрыться распорядиться похитить имущество JUR JUR JUR он же стульник с.в обвиняться в тот что находиться в бар расположить в в в период время с час до часы тайно похитить из карман куртка фио сотовый телефон стоимость рубль к т.е в совершение преступление предусмотренный год в судебный заседание прокурор астафьев е.о отказаться от дать часть обвинение стульников с.в в связь с чем суд вынести отдельный постановление от подсудимый стульник с.в вина в совершение преступление предусмотренный б признать в полный объём в содеять раскаяться и пояснить следующий в ночь на в поиск спиртное он идти по в к дом на который он указат

In [27]:
[i for i in df['tf-idf'][0] if i not in colloc_1000]

['фио пояснить судебный заседание следующий',
 'фио пояснить судебный заседание',
 'фио пояснить судебный',
 'пояснить судебный заседание следующий',
 'пояснить судебный заседание',
 'свидетель фио пояснить судебный заседание следующий он',
 'фио пояснить судебный заседание следующий он',
 'возмещение материальный ущерб',
 'ущерб сумма рубль',
 'пояснить судебный заседание следующий он',
 'свидетель фио пояснить судебный заседание следующий',
 'свидетель фио пояснить судебный заседание',
 'свидетель фио пояснить судебный',
 'совершение преступление предусмотренный',
 'признать полный объём содеять раскаяться',
 'признать полный объём содеять',
 'полный объём содеять раскаяться',
 'полный объём содеять',
 'объём содеять раскаяться',
 'сведение личность подсудимый',
 'признать полный объём',
 'чужое имущество действовать из корыстный побуждение',
 'чужое имущество действовать из корыстный',
 'чужое имущество действовать из',
 'чужое имущество действовать',
 'хищение чужое имущество дейст

In [ ]:
def del_jur(text):
    norm_text = ' '.join(normalize(text))
    for i in jur:
        norm_text = norm_text.replace(i, 'JUR')

In [52]:
from pymystem3 import Mystem

m = Mystem()
analysis = m.analyze(df['raw_main_parts'][0])

lemmas = ''
txt = ''
for item in analysis:
    if len(item) > 1:
        for i in item['analysis']:
            lemmas += i['lex'] + ' '
            txt += item['text'] + ' '